In [49]:
import os
import pandas as pd
pd.set_option('display.max_columns', None)

import geopandas as gpd

#library for connecting to the geodatabase
from sqlalchemy import create_engine, text
from geoalchemy2 import Geometry
import psycopg2

from shapely.geometry import Point
from shapely import wkb

In [63]:
# --- DB connection params ---
conn_params = {
    'host': 'PRIORPSRV03',
    'port': 5432,
    'dbname': 'gis',    
}

# --- Connect to Postgres ---
conn = psycopg2.connect(**conn_params)

In [64]:
import geopandas as gpd
from sqlalchemy import create_engine

# SQL query
query = """
SELECT *
FROM digital_place_portrait.urb_lnduse_cdrc_poi_july_2024
WHERE lad22nm = 'Leeds';
"""

# Load data into GeoDataFrame
cdrc_poi_gdf = pd.read_sql_query(query, conn)

# Preview
cdrc_poi_gdf.head()

C:\Users\abhimanya.achara\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,id,fid,primary_name,main_category,alternate_category,address,locality,postcode,region,country,source,source_record_id,lat,long,lsoa21nm,lsoa21cd,ni_dz2021cd,sg_dz2011cd,h3_15,lad22nm,wd21nm,geom,general_category
0,1763046,1879517,"""Cooper Drive""",None,None,"""17 Tawny Beck""","""Leeds""","""LS13 4UW""",None,"""GB""","""msft""","""1125899908650619""",-1.632455,53.797929,None,None,None,None,8f7bb1a34cb0d2d,Leeds,Pudsey,0101000020346C0000E77E633F3AE41941BE39AC9A0077...,None
1,1800952,1852630,"""Kingfisher Windows""","""retail""",None,"""Valley Mills, Whitehall Road""","""Bradford""","""BD11 1NQ""",None,"""GB""","""msft""","""844424930942097""",-1.645409,53.765384,None,None,None,None,8f7bb1a2650b329,Leeds,Morley North,0101000020346C00008800B6E82DD71941DC90C1785D3E...,None
2,1758358,1861561,"""Shedecks""",None,None,"""The Clearings""","""Leeds""","""LS10 3UU""",None,"""GB""","""msft""","""1125899908661623""",-1.535800,53.761900,None,None,None,None,8f7bb1a135acb6a,Leeds,Middleton Park,0101000020346C0000D9D945AA19481A4120739BAAF538...,None
3,1763494,1883909,"""K S P Consulting""",None,None,"""28-30 Gay Lane""","""Otley""","""LS21 1BR""",None,"""GB""","""msft""","""1125899908374059""",-1.690498,53.903036,None,None,None,None,8f7bb184acc9c32,Leeds,Otley & Yeadon,0101000020346C0000DB3FBD47B3A719418E0469946B2D...,None
4,1764155,1888886,"""Fluere""",None,None,"""15 Queen Square""","""Leeds""","""LS2 8AJ""",None,"""GB""","""msft""","""1125899908669981""",-1.544823,53.803573,None,None,None,None,8f7bb1aa5da3010,Leeds,Little London & Woodhouse,0101000020346C0000BEE677CB573E1A41822014EE5581...,None


In [65]:
cdrc_poi_gdf.drop(['fid','id','general_category','lsoa21nm','lsoa21cd','ni_dz2021cd','sg_dz2011cd','source','source_record_id','h3_15','country','region','geom'],1,inplace = True)
cdrc_poi_gdf.head()

C:\Users\abhimanya.achara\AppData\Local\Temp\ipykernel_22068\2002564371.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  cdrc_poi_gdf.drop(['fid','id','general_category','lsoa21nm','lsoa21cd','ni_dz2021cd','sg_dz2011cd','source','source_record_id','h3_15','country','region','geom'],1,inplace = True)


,primary_name,main_category,alternate_category,address,locality,postcode,lat,long,lad22nm,wd21nm
0,"""Cooper Drive""",None,None,"""17 Tawny Beck""","""Leeds""","""LS13 4UW""",-1.632455,53.797929,Leeds,Pudsey
1,"""Kingfisher Windows""","""retail""",None,"""Valley Mills, Whitehall Road""","""Bradford""","""BD11 1NQ""",-1.645409,53.765384,Leeds,Morley North
2,"""Shedecks""",None,None,"""The Clearings""","""Leeds""","""LS10 3UU""",-1.535800,53.761900,Leeds,Middleton Park
3,"""K S P Consulting""",None,None,"""28-30 Gay Lane""","""Otley""","""LS21 1BR""",-1.690498,53.903036,Leeds,Otley & Yeadon
4,"""Fluere""",None,None,"""15 Queen Square""","""Leeds""","""LS2 8AJ""",-1.544823,53.803573,Leeds,Little London & Woodhouse


In [66]:
fields_to_clean = ['primary_name', 'main_category', 'address', 'locality', 'postcode']

for field in fields_to_clean:
    cdrc_poi_gdf[field] = cdrc_poi_gdf[field].str.strip('"')

# Preview cleaned data
cdrc_poi_gdf.head()

,primary_name,main_category,alternate_category,address,locality,postcode,lat,long,lad22nm,wd21nm
0,Cooper Drive,None,None,17 Tawny Beck,Leeds,LS13 4UW,-1.632455,53.797929,Leeds,Pudsey
1,Kingfisher Windows,retail,None,"Valley Mills, Whitehall Road",Bradford,BD11 1NQ,-1.645409,53.765384,Leeds,Morley North
2,Shedecks,None,None,The Clearings,Leeds,LS10 3UU,-1.535800,53.761900,Leeds,Middleton Park
3,K S P Consulting,None,None,28-30 Gay Lane,Otley,LS21 1BR,-1.690498,53.903036,Leeds,Otley & Yeadon
4,Fluere,None,None,15 Queen Square,Leeds,LS2 8AJ,-1.544823,53.803573,Leeds,Little London & Woodhouse


In [67]:
# Create geometry column: Point(longitude, latitude)
cdrc_poi_gdf['geometry'] = cdrc_poi_gdf.apply(
    lambda row: Point(row['lat'], row['long']), axis=1
)

# Convert to GeoDataFrame
cdrc_poi_gdf = gpd.GeoDataFrame(cdrc_poi_gdf, geometry='geometry')

# Set the coordinate reference system (CRS) to WGS84 (EPSG:4326)
cdrc_poi_gdf.set_crs(epsg=4326, inplace=True)

cdrc_poi_gdf['id'] = range(1, len(cdrc_poi_gdf) + 1)

# Optional: preview
cdrc_poi_gdf.head()

,primary_name,main_category,alternate_category,address,locality,postcode,lat,long,lad22nm,wd21nm,geometry,id
0,Cooper Drive,None,None,17 Tawny Beck,Leeds,LS13 4UW,-1.632455,53.797929,Leeds,Pudsey,POINT (-1.63245 53.79793),1
1,Kingfisher Windows,retail,None,"Valley Mills, Whitehall Road",Bradford,BD11 1NQ,-1.645409,53.765384,Leeds,Morley North,POINT (-1.64541 53.76538),2
2,Shedecks,None,None,The Clearings,Leeds,LS10 3UU,-1.535800,53.761900,Leeds,Middleton Park,POINT (-1.53580 53.76190),3
3,K S P Consulting,None,None,28-30 Gay Lane,Otley,LS21 1BR,-1.690498,53.903036,Leeds,Otley & Yeadon,POINT (-1.69050 53.90304),4
4,Fluere,None,None,15 Queen Square,Leeds,LS2 8AJ,-1.544823,53.803573,Leeds,Little London & Woodhouse,POINT (-1.54482 53.80357),5


In [68]:
# Define database connection parameters
db_host = "PRIORPSRV03"
db_name = "gis"
db_port = "5432"
db_schema = "university_of_leeds"
table_name = "cdrc_points_of_interest"  # Desired table name
primary_key_column = "id"  # Define the primary key column (change based on your dataset)
geometry_column = "geometry"  # Default geometry column
# Create the database connection string (Windows Authentication - Trusted Connection)
conn_str = f"postgresql+psycopg2://@{db_host}:{db_port}/{db_name}?sslmode=disable"

# Create a SQLAlchemy engine
engine = create_engine(conn_str)

In [69]:
layer_to_load = cdrc_poi_gdf

In [70]:
# Ensure the GeoDataFrame has a valid CRS before writing
if layer_to_load.crs is None:
    print("Warning: GeoDataFrame has no CRS. Setting default to EPSG:27700 (British National Grid).")
    layer_to_load.set_crs(epsg=4326, inplace=True)

In [71]:
# Automatically detect geometry type from GeoDataFrame
geom_type = layer_to_load.geom_type.unique()[0].upper()
# Publish the GeoDataFrame to PostGIS
layer_to_load.to_postgis(
    name=table_name,
    con=engine,
    schema=db_schema,
    if_exists="replace",
    index=False,
    dtype = {'geometry': geom_type}
)

print(f"Data successfully uploaded to PostGIS: {db_schema}.{table_name}")

# Connect to the database to modify table structure
with engine.connect() as conn:
    # Set Primary Key (if it doesn't exist already)
    alter_pk_query = text(f"""
        ALTER TABLE {db_schema}.{table_name}
        ADD CONSTRAINT {table_name}_pkey PRIMARY KEY ({primary_key_column});
    """)
    
    # Create Spatial Index
    create_spatial_index_query = text(f"""
        CREATE INDEX {table_name}_geom_idx
        ON {db_schema}.{table_name}
        USING GIST ({geometry_column});
    """)

    try:
        conn.execute(alter_pk_query)  # Add Primary Key
        print(f"Primary key set on column: {primary_key_column}")
    except Exception as e:
        print(f"Could not set primary key. It may already exist. Error: {e}")

    try:
        conn.execute(create_spatial_index_query)  # Add Spatial Index
        print(f"Spatial index created for geometry column: {geometry_column}")
    except Exception as e:
        print(f"Could not create spatial index. It may already exist. Error: {e}")

print(f"GeoDataFrame successfully published to PostGIS with Primary Key and Spatial Index: {db_schema}.{table_name}")

Data successfully uploaded to PostGIS: university_of_leeds.cdrc_points_of_interest
Primary key set on column: id
Spatial index created for geometry column: geometry
GeoDataFrame successfully published to PostGIS with Primary Key and Spatial Index: university_of_leeds.cdrc_points_of_interest
